In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Bidirectional, Input

data = {
    "Hours": [1.2, 2.0, 3.5, 4.0, 5.3, 6.1, 7.2, 8.0, 9.5],
    "Scores": [18, 28, 33, 42, 57, 63, 77, 85, 97]
}

df = pd.DataFrame(data)
print("Sample Data:\n", df.head())

X = df[['Hours']].values
y = df['Scores'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.35, random_state=123)

models = {
    "Linear Regression": LinearRegression(),
    "SVM": SVR(kernel='rbf', C=1.2, epsilon=0.2),
    "Random Forest": RandomForestRegressor(n_estimators=120, max_depth=5, random_state=123),
    "XGBoost": XGBRegressor(objective='reg:squarederror', n_estimators=120, learning_rate=0.12, random_state=123),
    "KNN": KNeighborsRegressor(n_neighbors=4),
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    results[name] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

#Naive Bayes
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
nb = GaussianNB()
y_train_int = np.round(y_train)
nb.fit(X_train_scaled, y_train_int)
preds = nb.predict(X_test_scaled)
results["Naive Bayes (approx)"] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

mlp = MLPRegressor(hidden_layer_sizes=(70,30), max_iter=1800, random_state=123)
mlp.fit(X_train, y_train)
preds = mlp.predict(X_test)
results["MLP (Sklearn)"] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

#Deep Learning Model
X_train_rnn = np.expand_dims(X_train, axis=1)
X_test_rnn = np.expand_dims(X_test, axis=1)

def build_and_train(model, name):
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_rnn, y_train, epochs=120, verbose=0)
    preds = model.predict(X_test_rnn).flatten()
    results[name] = {"MSE": mean_squared_error(y_test, preds), "R2": r2_score(y_test, preds)}

rnn = Sequential([Input(shape=(1,1)), SimpleRNN(28), Dense(1)])
build_and_train(rnn, "RNN")

lstm = Sequential([Input(shape=(1,1)), LSTM(28), Dense(1)])
build_and_train(lstm, "LSTM")

bilstm = Sequential([Input(shape=(1,1)), Bidirectional(LSTM(28)), Dense(1)])
build_and_train(bilstm, "BiLSTM")

gru = Sequential([Input(shape=(1,1)), GRU(28), Dense(1)])
build_and_train(gru, "GRU")

bigru = Sequential([Input(shape=(1,1)), Bidirectional(GRU(28)), Dense(1)])
build_and_train(bigru, "BiGRU")


print("\nModel Performance (MSE & R2):")
print(pd.DataFrame(results).T)




Sample Data:
    Hours  Scores
0    1.2      18
1    2.0      28
2    3.5      33
3    4.0      42
4    5.3      57
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 599ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 642ms/step

Model Performance (MSE & R2):
                              MSE        R2
Linear Regression        5.874146  0.991248
SVM                    922.643022 -0.374643
Random Forest          109.817344  0.836384
XGBoost                330.158344  0.508098
KNN                    419.296875  0.375291
Naive Bayes (approx)   170.000000  0.746718
MLP (Sklearn)            8.277894  0.987667
RNN                   3710.210857 -4.527831
LSTM                  4116.265323 -5.132810
BiLSTM                3864.072093 -4.757068
GRU                   3863.803679 -4.756668
BiGRU                 3601.488976 -4.365846
